# Step 2 - Data Wrangling Raw Data in Local Data Lake to Digestable Data  
Loading, merging, cleansing, unifying and wrangling Oracle OpenWorld & CodeOne Session Data from still fairly raw JSON files in the local datalake. 

The gathering of raw data from the (semi-)public API for the Session Catalog into a local data lake was discussed and performed  in <a href="./1-OOW2018 Session Catalog - retrieving raw session data in JSON files.ipynb">Notebook 1-OOW2018 Session Catalog - retrieving raw session data in JSON files</a>. The current notebook starts from th 44 raw JSON files in local directory `./data`.

This notebook describes how to load, combine and wrangle the data from these files. This notebook shows for example how to load and merge data from dozens of (same formatted) JSON files, discard undesired attributes, deduplicate the record set, derive attributes for easier business intelligence & machine learning and write the resulting data set to a single JSON file.

Steps in this notebook:
* Load and Merge from raw JSON
    * <a href="#deduplicate">discard redundant columns</a>
    * <a href="#deduplicate">deduplication</a> 
* <a href="#explore">Explore Data Frame</a>
* <a href="#enriching">Enrich Data</a>
* Publish Wrangle Results

# Load and merge data from raw JSON files
This first section describes how the session data from Oracle OpenWorld 2018 is loaded from over 40 individual files with the raw JSON session data. These files are organized by session type and event (oow and codeone) - and have been produced by a different notebook (<a href="./1-OOW2018 Session Catalog - retrieving raw session data in JSON files.ipynb">Notebook 1-OOW2018 Session Catalog - retrieving raw session data in JSON files</a>) from the Oracle OpenWorld Session Catalog API.

The files are read into individual Pandas Data Frame objects. These Data Frames are concatenated. The end result from reading 44 files is a single Pandas Data Frame - called *ss* (session schedule).

Let's start with reading the session information from a single file into a Pandas Data Frame - to get a feel for how that works and what it results in.

In [11]:
#read a single session data file and parse the JSON content into a Pandas Data Frame
import pandas as pd
import json

dataLake = "datalake/" # file system directory used for storing the gathered data

#as a test, try to load data from one of the generated files 
conference = 'oow' # could also be codeone
sessionType = 'TRN' # could also be one of 21 other values such as TUT, DEV, GEN, BOF, HOL, ...
ss = pd.read_json("{0}oow2018-sessions_{1}_{2}.json".format(dataLake, conference, sessionType))
# add an additional column to the Data Frame to specify the conference catalog of origin of these sessions; in this case oow
ss = ss.assign(catalog=conference)

ss.head(3)


,abbreviation,abstract,allowDoubleBooking,attributevalues,code,codeParts,code_id,es_metadata_id,event,eventCode,...,type_displayorder_string,useDoubleBooking,useWaitingList,videos,viewAccess,viewAccessPublic,viewFileAccess,waitlistAccess,waitlistLimit,catalog
0,TRN4070,What if you could benefit from new database qu...,0,"[{'value': 'Beginner', 'attributevalue_id': '1...",TRN4070,"{'alpha0': 'TRN', 'numeric1': '4070'}",trn4070,1523650128750001VI1i,Oracle OpenWorld,oow18,...,9999,True,0,[],"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0,oow
1,TRN4026,How does a real-world performance engineer int...,0,"[{'value': 'Beginner', 'attributevalue_id': '1...",TRN4026,"{'alpha0': 'TRN', 'numeric1': '4026'}",trn4026,1523576222383001pdRl,Oracle OpenWorld,oow18,...,9999,True,0,[],"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0,oow
10,TRN4567,Understand the tools and techniques available ...,0,"[{'value': 'Beginner', 'attributevalue_id': '1...",TRN4567,"{'alpha0': 'TRN', 'numeric1': '4567'}",trn4567,1524514304383001hANs,Oracle OpenWorld,oow18,...,9999,True,0,[],"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0,oow


In [12]:
#as a test, try to load data from another file ; same sessionType but different conference 

conference = 'codeone' 
sessionType = 'TRN'
ss2 = pd.read_json("{0}oow2018-sessions_{1}_{2}.json".format(dataLake, conference, sessionType))
# add an additional column to the Data Frame to specify the conference catalog of origin of these sessions; in this case codeone
ss2 = ss2.assign(catalog='codeone')

ss2.head(3)


,abbreviation,abstract,allowDoubleBooking,attributevalues,code,codeParts,code_id,es_metadata_id,event,eventCode,...,type_displayorder_string,useDoubleBooking,useWaitingList,videos,viewAccess,viewAccessPublic,viewFileAccess,waitlistAccess,waitlistLimit,catalog
0,TRN4631,This session takes a deep dive into how the Or...,0,"[{'value': 'Beginner', 'attributevalue_id': '1...",TRN4631,"{'alpha0': 'TRN', 'numeric1': '4631'}",trn4631,1524589840877001mdKy,Oracle OpenWorld,oow18,...,9999,True,0,None,"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0,codeone
1,TRN4081,Real-time data is an increasingly popular topi...,0,"[{'value': 'Beginner', 'attributevalue_id': '1...",TRN4081,"{'alpha0': 'TRN', 'numeric1': '4081'}",trn4081,1523652232625001KxHD,Oracle OpenWorld,oow18,...,9999,True,0,[],"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0,codeone
10,TRN4084,.NET Core allows .NET applications to run cros...,0,"[{'value': 'Beginner', 'attributevalue_id': '1...",TRN4084,"{'alpha0': 'TRN', 'numeric1': '4084'}",trn4084,1523652635507001BgE3,Oracle OpenWorld,oow18,...,9999,True,0,[],"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0,codeone


All session data is to be merged into a single data frame. We will use `ss` as the sink - the data frame into which all sessions records are to be loaded. We use the Pandas concat operation to merge two Data Frames, as is done below for the two data frames with session records for session type TRN.

In [13]:
#add ss and ss2 together
#see https://pandas.pydata.org/pandas-docs/stable/merging.html
ss = pd.concat([ss,ss2],  ignore_index=True , sort=True)
ss.head(8)

,abbreviation,abstract,allowDoubleBooking,attributevalues,catalog,code,codeParts,code_id,es_metadata_id,event,...,type_displayorder,type_displayorder_string,useDoubleBooking,useWaitingList,videos,viewAccess,viewAccessPublic,viewFileAccess,waitlistAccess,waitlistLimit
0,TRN4070,What if you could benefit from new database qu...,0,"[{'value': 'Beginner', 'attributevalue_id': '1...",oow,TRN4070,"{'alpha0': 'TRN', 'numeric1': '4070'}",trn4070,1523650128750001VI1i,Oracle OpenWorld,...,9999,9999,True,0,[],"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0
1,TRN4026,How does a real-world performance engineer int...,0,"[{'value': 'Beginner', 'attributevalue_id': '1...",oow,TRN4026,"{'alpha0': 'TRN', 'numeric1': '4026'}",trn4026,1523576222383001pdRl,Oracle OpenWorld,...,9999,9999,True,0,[],"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0
2,TRN4567,Understand the tools and techniques available ...,0,"[{'value': 'Beginner', 'attributevalue_id': '1...",oow,TRN4567,"{'alpha0': 'TRN', 'numeric1': '4567'}",trn4567,1524514304383001hANs,Oracle OpenWorld,...,9999,9999,True,0,[],"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0
3,TRN6363,In order to derive value from the Internet of ...,0,"[{'value': 'Product Training Session', 'attrib...",oow,TRN6363,"{'alpha0': 'TRN', 'numeric1': '6363'}",trn6363,15321279159350012tnD,Oracle OpenWorld,...,9999,9999,True,0,[],"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0
4,TRN5828,Oracle Linux powers tens of thousands of enter...,0,"[{'value': 'Beginner', 'attributevalue_id': '1...",oow,TRN5828,"{'alpha0': 'TRN', 'numeric1': '5828'}",trn5828,1526317377636001I2Ll,Oracle OpenWorld,...,9999,9999,True,0,[],"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0
5,TRN4073,"What do Oracle Database listener, dedicated se...",0,"[{'value': 'Beginner', 'attributevalue_id': '1...",oow,TRN4073,"{'alpha0': 'TRN', 'numeric1': '4073'}",trn4073,1523650870534001B9V8,Oracle OpenWorld,...,9999,9999,True,0,None,"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0
6,TRN4022,Oracle Real Application Clusters (Oracle RAC) ...,0,"[{'value': 'Beginner', 'attributevalue_id': '1...",oow,TRN4022,"{'alpha0': 'TRN', 'numeric1': '4022'}",trn4022,1523572455776001BxBi,Oracle OpenWorld,...,9999,9999,True,0,[],"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0
7,TRN4219,"For more than 20 years, Oracle customers have ...",0,"[{'value': 'Beginner', 'attributevalue_id': '1...",oow,TRN4219,"{'alpha0': 'TRN', 'numeric1': '4219'}",trn4219,152392541435100177jo,Oracle OpenWorld,...,9999,9999,True,0,[],"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0


Overhead, two files were loaded, parsed into a Data Frame and added together into a single Data Frame. The next step is to load the session data from the raw JSON file for all 44 files - for the two events and for all session types. The session types are defined in the Dict object *sessionTypes* . The code loops over the keys in this Dict and reads the corresponding JSON file for each of the two events.

In [15]:
sessionTypes =  {'BOF': '1518466139979001dQkv'
    , 'BQS': 'bqs'
    , 'BUS': '1519240082595001EpMm'
    , 'CAS': 'casestudy'
    , 'DEV': '1522435540042001BxTD'
    , 'ESS': 'ess'
    , 'FLP': 'flp'
    , 'GEN': 'general'
    , 'HOL': 'hol'
    , 'HOM': 'hom'
    , 'IGN': 'ignite'
    , 'KEY': 'option_1508950285425'
    , 'MTE':'1523906206279002QAu9'
    , 'PKN': '1527614217434001RBfj'
    , 'PRO': '1518464344082003KVWZ'
    , 'PRM': '1518464344082002KM3k'
    , 'TRN': '1518464344082001KHky'
    , 'SIG': 'sig'
    , 'THT': 'ts'
    , 'TLD': '1537894888625001RriS'
    , 'TIP': '1517517756579001F3CR'
    , 'TUT': 'tutorial'
#commented out because TRN is dealt with earlier on, 'TRN': '1518464344082001KHky'
}
#loop over all session types and read the corresponding files for both events codeone and oow
for key,value in sessionTypes.items():
    sessionType = key
    conference = 'oow' 
    ssoow = pd.read_json("{0}oow2018-sessions_{1}_{2}.json".format(dataLake, conference, sessionType))
    # add an additional column to the Data Frame to specify the conference catalog of origin of these sessions
    ssoow = ssoow.assign(catalog=conference)
    
    conference = 'codeone' 
    sscodeone = pd.read_json("{0}oow2018-sessions_{1}_{2}.json".format(dataLake, conference, sessionType))
    sscodeone = sscodeone.assign(catalog=conference)
    
    # merge data for sessions of type session type for both oow and codeone into a master set in ss
    ss = pd.concat([ss,ssoow,sscodeone],  ignore_index=True, sort=True)
    
print("Done - all data is merged into one data frame")    

Done - all data is merged into one data frame


### Some key  metrics on the merged Sessions Set
The shape function on the data frame returns the dimensions of the frame: the number of rows by the number of columns:

In [26]:
ss.shape

(3836, 39)

In [25]:
# total memory usage
ss.memory_usage(index=True, deep=True).sum()

13158273

In [162]:
#list all columns in the Dataframe
ss.columns

Index(['abbreviation', 'abstract', 'allowDoubleBooking', 'attributevalues',
       'catalog', 'code', 'codeParts', 'code_id', 'es_metadata_id', 'event',
       'eventCode', 'eventId', 'eventName', 'externalID', 'featured_value',
       'files', 'highlight', 'length', 'participants',
       'publicViewPrivateSchedule', 'published', 'scheduleAccess', 'sessionID',
       'sponsors', 'status', 'times', 'title', 'title_sort', 'type',
       'type_displayorder', 'type_displayorder_string', 'useDoubleBooking',
       'useWaitingList', 'videos', 'viewAccess', 'viewAccessPublic',
       'viewFileAccess', 'waitlistAccess', 'waitlistLimit'],
      dtype='object')

In [29]:
# data types for the columns in the data frame
ss.dtypes

abbreviation                  object
abstract                      object
allowDoubleBooking           float64
attributevalues               object
catalog                       object
code                          object
codeParts                     object
code_id                       object
es_metadata_id                object
event                         object
eventCode                     object
eventId                       object
eventName                     object
externalID                    object
featured_value               float64
files                         object
highlight                     object
length                       float64
participants                  object
publicViewPrivateSchedule     object
published                    float64
scheduleAccess                object
sessionID                     object
sponsors                      object
status                        object
times                         object
title                         object
t

<a name="discard" />

## Discard unneeded attributes
Early inspection of the JSON document and the session catalog website has provided insight in the available attributes and their relevance. Below, you will see an overview of all columns in the Data Frame - corresponding with the top level items in the JSON document. The subsequent step removes from the Data Frame all columns that seem irrelevant for our task at hand. 

These columns seem relevant for the web frontend developers, for planned but not realized objectives or for unknown purposes. In order to not carry more weight than necessary - because of performance, resource usage and lurking complexity we get rid of columns that seem irrelevant. If we need them after all, they are still available in the data lake.    

Remove the unwanted columns from the Dataframe
(allowDoubleBooking .'codeParts', 'code_id', 'es_metadata_id',type_displayorder	type_displayorder_string	useDoubleBooking	useWaitingList	videos	viewAccess	viewAccessPublic, ...)


In [163]:
# remove columns
# Note that the original 'data' object is changed when inplace=True
ss.drop([
    'allowDoubleBooking'
         ,'codeParts', 'code_id', 'es_metadata_id','type_displayorder'
       ,'type_displayorder_string', 'useDoubleBooking'
       ,'useWaitingList', 'videos'
      , 'viewAccess', 'viewAccessPublic','viewFileAccess', 'waitlistAccess', 'waitlistLimit'
    ,  'eventId','eventName','featured_value','publicViewPrivateSchedule','published', 'scheduleAccess','sessionID','status'
    ,'externalID','highlight','abbreviation'
        ]
        , axis=1, inplace=True) 

<a name="deduplicate" />

## Deduplicate
Some sessions are included in the catalog for both events - Oracle OpenWorld and CodeOne - even though they are associated with one of the two. The exact same session - with only a different value for attribute catalog - occurs twice in our data set for these sessions. We should get rid of duplicates. However, we should not do so before we capture the fact that a session is part of the catalogs of both events in the record that is retained. 

Note: it seems there are some sessions that occur multiple times in the data set but are not included in both event catalogs. This seems to be just some form of data pollution.

Let's first see how many events are part of both events' catalogs. 

In [80]:
# The code feature is supposed to be the unique identifier of sessions
# Let's see at the multifold occurrence of individual code values

counts = ss['code'].value_counts()
counts.head(13)

TRN5777    6
TRN4092    6
TRN4077    6
TRN4081    6
TRN4019    6
TRN4065    6
TRN4088    6
TRN5774    6
TRN4631    6
TRN4082    6
TRN4084    6
TRN5793    6
TIP4175    4
Name: code, dtype: int64

We have found quite a few code values that occur multiple times in the data frame. Each code should be in the data frame only once. Let's further look into these sessions.

In [84]:
# let's create a data frame with all sessions whose code occurs more than one in the data frame
duplicates = ss[ss['code'].isin(counts.index[counts > 1])]
# show the first ten records of these 'candidate duplicates'
duplicates[['code','title','event','catalog'  ]].sort_values('code').head(10)

,code,title,event,catalog
178,BOF4622,OpenMessaging: New Messaging Standard Under th...,Oracle Code One,codeone
2023,BOF4622,OpenMessaging: New Messaging Standard Under th...,Oracle Code One,codeone
191,BOF4737,Designing APIs for Cloud Infrastructure with E...,Oracle Code One,codeone
2036,BOF4737,Designing APIs for Cloud Infrastructure with E...,Oracle Code One,codeone
187,BOF4759,Writing Better Code (Faster) in the Apache Net...,Oracle Code One,codeone
2032,BOF4759,Writing Better Code (Faster) in the Apache Net...,Oracle Code One,codeone
2002,BOF4760,Hacking the NetBeans IDE,Oracle Code One,codeone
157,BOF4760,Hacking the NetBeans IDE,Oracle Code One,codeone
2037,BOF4817,Diversity and Inclusion: Are We There Yet?,Oracle Code One,codeone
192,BOF4817,Diversity and Inclusion: Are We There Yet?,Oracle Code One,codeone


In [184]:
#are duplicates indeed associated with both events?
# some are - but not all of them:
duplicates.loc[duplicates['code']=='BOF4977']


,abstract,attributevalues,catalog,code,event,eventCode,files,length,participants,sponsors,...,Beginner,Advanced,All,track,speaker_count,instance_count,room_capacity,day,room,time
147,Microservices are independent—sure. Complex tr...,"[{'value': 'Intermediate', 'attributevalue_id'...",oow,BOF4977,Oracle OpenWorld,oow18,"[{'fileId': '1540345180625001DgUR', 'filename'...",45.0,[{'speakerId': '14574694420080015I8e_150393630...,None,...,N,N,N,"[Containers, Serverless, and Cloud, Infrastruc...",1,1.0,303,Tuesday,Moscone West - Room 2006,19:30
151,Microservices are independent—sure. Complex tr...,"[{'value': 'Intermediate', 'attributevalue_id'...",codeone,BOF4977,Oracle OpenWorld,oow18,"[{'fileId': '1540345180625001DgUR', 'filename'...",45.0,[{'speakerId': '14574694420080015I8e_150393630...,None,...,N,N,N,"[Containers, Serverless, and Cloud, Infrastruc...",1,1.0,303,Tuesday,Moscone West - Room 2006,19:30


The next step is a little bit complex: we want to record the fact that certain sessions (actually session codes) are associated with both catalogs. We join the sessions in `ss` with the sessions that occur multiple times in `duplicates` and we join records in `ss` with their counterparts (same session code) that have a different catalog origin.

This gives us a data frame with all session codes associated with both catalogs.

In [85]:
# find all sessions that occur in both catalogs: set their catalog attribute to both
# set catalog="both" if session in duplicates with a different catalog value than the session's own catalog value
doubleCatalogSessions = pd.merge(ss, duplicates, on=['code'], how='inner').query('catalog_y != catalog_x')
doubleCatalogSessions[['code','catalog_x', 'catalog_y']] .head(20)



,code,catalog_x,catalog_y
100,TRN5793,oow,codeone
102,TRN5793,oow,codeone
104,TRN5793,oow,codeone
105,TRN5793,codeone,oow
107,TRN5793,codeone,oow
109,TRN5793,codeone,oow
112,TRN5793,oow,codeone
114,TRN5793,oow,codeone
116,TRN5793,oow,codeone
117,TRN5793,codeone,oow


The master dataset is still `ss`. All sessions in this data frame whose session code appears in `doubleCatalogSessions` will get their *catalog* attribute updated to *both*. The cell will then show the values in catalog and the number of their occurrences.

In [87]:
# all sessions in doubleCatalogSessions occur in both oow and code one session catalog
# time to update column catalog for all sessions in ss that have a code that occurs in doubleCatalogSessions['code']
ss.loc[ss['code'].isin(doubleCatalogSessions['code']),'catalog']='both'
ss['catalog'].value_counts()

oow        3062
both        444
codeone     330
Name: catalog, dtype: int64

If we now drop any duplicate records - any sessions whose session code occurs more than once - we will reduce our data frame to the unique set of sessions that actually took place, without the ghost duplicates introduced in our data set because a session appeared in more than one catalog.

In [88]:
#Drop duplicates - identifying rows by their code
ss.drop_duplicates(subset=['code'], keep='first', inplace=True)

In [89]:
#hpw many sessions appear in each and in both catalogs?
ss['catalog'].value_counts()

oow        1470
codeone     165
both        105
Name: catalog, dtype: int64

<a name="explore" />

# Exploring the Data
Let's briefly look at the data we now have in the Pandas Data Frame. What does the data look like? What values do we have in the columns? And what complexity is hiding in some columns with nested values, such as participants, attributevalues and files.

Note: https://morphocode.com/pandas-cheat-sheet/ provides a quick overview of commands used for inspecting and manipulating the data frame.

In [164]:
#an overview of the current contents of the data frame
ss.head(6)


,abstract,attributevalues,catalog,code,event,eventCode,files,length,participants,sponsors,times,title,title_sort,type
0,What if you could benefit from new database qu...,"[{'value': 'Beginner', 'attributevalue_id': '1...",oow,TRN4070,Oracle OpenWorld,oow18,"[{'fileId': '1540594881063001PjdE', 'filename'...",45.0,[{'speakerId': '14574694377280015QNA_150393630...,None,"[{'sessionTimeID': '1532616475210001usPa', 'se...",A Database Proxy for Transparent High Availabi...,adatabaseproxyfortransparenthi,Product Training Session
1,How does a real-world performance engineer int...,"[{'value': 'Beginner', 'attributevalue_id': '1...",oow,TRN4026,Oracle OpenWorld,oow18,"[{'fileId': '1540053090863001JeFM', 'filename'...",45.0,[{'speakerId': '145746969078000151fg_150393630...,None,"[{'sessionTimeID': '1531945554973001z4Hi', 'se...",A Day in the Life of a Real-World Performance ...,adayinthelifeofarealworldperfo,Product Training Session
2,Understand the tools and techniques available ...,"[{'value': 'Beginner', 'attributevalue_id': '1...",oow,TRN4567,Oracle OpenWorld,oow18,"[{'fileId': '1539988618000001VcCe', 'filename'...",45.0,[{'speakerId': '1468532302119001DtNf_150393630...,None,"[{'sessionTimeID': '15352384203600012G04', 'se...",Best Practices for Diagnosing and Troubleshoot...,bestpracticesfordiagnosingandt,Product Training Session
3,In order to derive value from the Internet of ...,"[{'value': 'Product Training Session', 'attrib...",oow,TRN6363,Oracle OpenWorld,oow18,"[{'fileId': '1539982921144001RL4h', 'filename'...",45.0,[{'speakerId': '1500580326390001qTtd_150393630...,None,"[{'sessionTimeID': '1535223010412001BrNh', 'se...",Oracle Internet of Things Cloud Service and En...,oracleinternetofthingscloudser,Product Training Session
4,Oracle Linux powers tens of thousands of enter...,"[{'value': 'Beginner', 'attributevalue_id': '1...",oow,TRN5828,Oracle OpenWorld,oow18,"[{'fileId': '1540152941675001kZHw', 'filename'...",45.0,[{'speakerId': '14574694616420015a7T_150393630...,None,"[{'sessionTimeID': '1535040668257001twFq', 'se...",Oracle Linux and Oracle VM: Get Trained for Cl...,oraclelinuxandoraclevmgettrain,Product Training Session
5,"What do Oracle Database listener, dedicated se...","[{'value': 'Beginner', 'attributevalue_id': '1...",oow,TRN4073,Oracle OpenWorld,oow18,None,45.0,[{'speakerId': '14574694719240015Rek_150393630...,None,"[{'sessionTimeID': '1531848510829001f1BX', 'se...",Oracle Net Services: Best Practices for Databa...,oraclenetservicesbestpractices,Product Training Session


In [165]:
# and this overview of the rows and columns in the data frame.
print(ss.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2339 entries, 0 to 2338
Data columns (total 14 columns):
abstract           2339 non-null object
attributevalues    2339 non-null object
catalog            2339 non-null object
code               2339 non-null object
event              2339 non-null object
eventCode          2339 non-null object
files              1615 non-null object
length             2339 non-null float64
participants       2339 non-null object
sponsors           244 non-null object
times              2338 non-null object
title              2339 non-null object
title_sort         2339 non-null object
type               2339 non-null object
dtypes: float64(1), object(13)
memory usage: 255.9+ KB
None


In [166]:
#let's look at all different values for length (the duration of each session in minutes)
ss['length'].unique()
# yes - it is that simple!

array([  45.,   20.,  240.,  180.,   90.,  120.,   75.,   60.,   30.,
        105.,   50.,   25.,   55.,   15.])

In [167]:
# and what about (session) type?
ss['type'].unique()

array(['Product Training Session', 'Birds of a Feather (BOF) Session',
       'Burning Questions Session', 'Business Use Case Session',
       'Customer Case Study Session', 'Developer Session',
       'Executive Solution Session', 'Flipped Session', 'General Session',
       'HOL (Hands-on Lab) Session', 'Home Room Session', 'Ignite Session',
       'Keynote Session', 'Meet the Experts', 'Product Keynote Session',
       'Product Overview Session', 'Product Roadmap Session',
       'Special Interest Group (SIG) Meeting', 'Theater Session',
       'Thought Leader Session', 'Tips and Tricks Session',
       'Tutorial Session'], dtype=object)

Some of the columns in the data frame contain complex, nested values. For example the `attributevalues` column. It contains a JSON array - a list of objects that each describe some attribute for the session. Examples of session attributes that are defined in this somewhat convoluted wat are *(target Experience) Level*, *Track*, *Day* , *Role*. A little later on, we will create new, proper features in the data frame based on values extracted from this complex attributevalues colunmn - in order to make it possible to make good use of this information for visualization, analysis and machine learning.

In [33]:
# some of the columns or attributes have nested values. It seems useful to take a closer look at them. 
# show nested array attributevalues for first record in Dataframe - this is an attribute which contains an array of objects that each define a specific characteristic of the session, 
# such as date and time, track, topic, level, role, company market size, 

ss['attributevalues'][10]

[{'value': 'Beginner',
  'attributevalue_id': '1486682599762002N05j',
  'rf_attributevalue_id': '1486682600237001NDhX',
  'attributevalue_code': '1486682599762002N05j',
  'dataForm': '1503936308L8ef50df64',
  'published': 1.0,
  'viewAccessPublic': True,
  'viewAccess': ['1533140082245001une2',
   '1533141196635001EZdJ',
   '1533143193464001uQsQ',
   '1533146325902001u9v6'],
  'scheduleAccess': ['1533141196635001EZdJ', '1533143193464001uQsQ'],
  'attribute_id': 'SessionsbyExperienceLevel',
  'attribute': 'Sessions by Experience Level',
  'displayorder': 1.0},
 {'value': 'Intermediate',
  'attributevalue_id': '1486682599762003NG3N',
  'rf_attributevalue_id': '1486682600237002N7TO',
  'attributevalue_code': '1486682599762003NG3N',
  'dataForm': '1503936308L8ef50df64',
  'published': 1.0,
  'viewAccessPublic': True,
  'viewAccess': ['1533140082245001une2',
   '1533141196635001EZdJ',
   '1533143193464001uQsQ',
   '1533146325902001u9v6'],
  'scheduleAccess': ['1533141196635001EZdJ', '153314

The *participants* column also contains a complex object. This column too contains a JSON array with the people associated with a session as speaker. The array contains a nested object for each speaker. This object has a lot of data in it - from name and biography and special designations (titles) for the speaker to company, job title, URL to a picture and details on all (other) sessions the speaker is involved in.

Let's take a look at an example.

In [169]:
#show nested array participants for 11th record in the Dataframe
ss['participants'][10]


[{'Speaker-Photo-Published': 'Published',
  'attributevalues': [{'attribute': 'Special Designations',
    'attribute_id': 'specialdesignations',
    'attributevalue_id': 'sdoraclecertified',
    'dataForm': '1503936308F8c7e51a2e',
    'value': 'Oracle Certified'},
   {'attribute': 'Special Designations',
    'attribute_id': 'specialdesignations',
    'attributevalue_id': 'sdjavaonerockstar',
    'dataForm': '1503936308F8c7e51a2e',
    'value': 'JavaOne Rockstar'}],
  'bio': 'My name is Ron Soltani, and I am a Sr. Principal Instructor at Oracle University.  I have been with Oracle University for over 14 Years teaching, Oracle Database, MYSQL, and Times Ten Database courses and I am the lead instructor for Oracle Database New Features, Oracle Database Security, and MYSQL Administration courses.  I am based in Southern Oregon and enjoy spending my spare time outdoors and working on our farm. ',
  'companyName': 'Oracle',
  'displayorder': 0.0,
  'es_metadata_id': '1500578268510001KsST_150

The *files* column too contains a JSON array. This array contains entries for files associated with the session. These files provide the slides or other supporting content for the session. Each session is supposed to have exactly one files associated with it. Some do not - for example because the speaker(s) forgot to upload their slides. 

Let's inspect the JSON contents of the *files* column. It contains the name of the file and the URL from where it can be downloaded.

In [35]:
#show nested array files
ss['files'][0]

[{'fileId': '1540594881063001PjdE',
  'filename': 'TRN4070_A-Database-Proxy-for-Transparent-High-Availability-Performance-Routing-and-Security.pdf',
  'url': 'https://static.rainfocus.com/oracle/oow18/sess/1523650128750001VI1i/PF/TRN4070_A-Database-Proxy-for-Transparent-High-Availability-Performance-Routing-and-Security_1540594881049001PwHW.pdf',
  'viewFileAccess': []}]

<a name="enriching" />

# Enriching the data
In this section we are engineering the data to produce some attributes or features that are easier to work with once we start doing data science activities such as business intelligence or machine learning. In this section we are not yet actually bringing in external data sources to add information we not already have in our set. However, we are making the data we already have more accessible and thereby more valuable. So in that sense, this too can be called *enriching*. 

The enrichments performed on the session data in the data frame :
* set a flag at session level to indicate whether the session is an Oracle session (with Oracle staff among the speakers- speaker has Oracle in companyName attribute)
* set a flag at session level to indicate whether a file has been uploaded for the session
* set a flag at session level to indicate whether one of the speakers has Java Rockstar, Java Champion, Developer Champion/Groundbreaker Ambassador, ACE or ACE Director
* set attribute level (based on attributevalues array) - beginner, all, ...
* derive track(s?) from attributevalues array (where attribute_id=Track or attribute_id=CodeOneTracks  )
* set attribute with number of speakers on session
* number of instances of the session    

### Oracle Speaker and File Uploaded
These functions `oracle_speaker` and `file_flag` are invoked for every record in the data frame. They are used to derive new, first class attributes to indicate whether or not at least one speaker working for Oracle is associated with a session (Y or N) and if a file has been upload (presumably with slides) for the session. The information represented by these two new attributes already exists in the data frame - but in a way that makes it quite unaccessible to the data analyst.

In [37]:
#function to derive Oracle flag from participants
def oracle_speaker(session):
      result = "N"
      # loop over speakers; if for any of them, companyName contains the word Oracle, the result =Y  
      for x in session["participants"][:]:
        if ("oracle" in x.get('companyName','x').lower()):
          result='Y'
      return result

In [38]:
#function to derive file flag from files
def file_flag(session):
      result = "N"
      if type(session.get("files",None)) =='list' :
          # loop over files; if any exist, then result = Y
          for x in session["files"][:]:
             if x['fileId']:
               result='Y'
      return result

New columns `oracle_speaker` and `file_flag` are added to the data frame with values derived for each record by applying the functions with corresponding names.  

In [40]:
#set oracle_speaker flag
#apply function oracle_speaker to every row in the data frame to derive values for the new column oracle_speaker
ss['oracle_speaker'] = ss.apply(oracle_speaker, axis=1)

#set file_flag
#apply function file_flag to every row in the data frame to derive values for the new column file_flag
ss['file_flag'] = ss.apply(file_flag, axis=1)


# show the values and the number of occurrences for the new column oracle_speaker
ss['oracle_speaker'].value_counts()


Y    2959
N     877
Name: oracle_speaker, dtype: int64

### Speaker Designations
Many of the speakers are special - in the sense that they have been awarded community awards and titles, such as (Oracle) Java Champion, Oracle ACE Directory, JavaOne Rockstar and Groundbreaker Ambassador. These designations can be found for speakers (a nested JSON object) in their *attributevalues* feature - which happens to be another nested JSON object.

The next function *speaker_designation* finds out for a session if it has at least one speaker associated with it who has the requested designation.

In [41]:
#function to derive designation flag from speakers
# values for designation: JavaOne Rockstar, Oracle ACE Director, Oracle Java Champion, Groundbreaker Ambassador, 
def speaker_designation(session, designation):
      result = "N"
      # loop over speakers and their attributevalues; if any exist with attribute_id == specialdesignations and value == JavaOne Rockstar
      for x in session["participants"][:]:
          if "attributevalues" in x: 
              for y in x["attributevalues"][:]:
                if "attribute_id" in y: 
                    if y["attribute_id"]=="specialdesignations":
                        if y["value"]== designation:
                            result="Y"
      return result

The next cell iterates over four major `designations` and derives for each designation a new column in the data frame that contain Y or N, depending on whether a speaker with the designation will present in the session. 

In [50]:
#set flags for designations
designations = ['JavaOne Rockstar', 'Oracle ACE Director', 'Oracle Java Champion', 'Groundbreaker Ambassador']
for d in designations:
   ss[d] = ss.apply(speaker_designation, args=(d,), axis=1) 

Let's check the newly created columns: what values do they contain and how often does each value occur?

In [49]:
ss[['JavaOne Rockstar','Oracle ACE Director' , 'Oracle Java Champion', 'Groundbreaker Ambassador']].apply(pd.value_counts).fillna(0)

,JavaOne Rockstar,Oracle ACE Director,Oracle Java Champion,Groundbreaker Ambassador
N,3697,3637,3740,3690
Y,139,199,96,146


### Level
Each session can be described as suitable for one or more levels of experience: Beginner, Intermediate or Advaned. Each session can be associated with all levels - or a subset of them. This level indication is somewhat hidden away, in the attributevalues object. The next function `session_level` will unearth for a given session whether it is associated with the specified level, Y or N.

In [55]:
#function to derive level flag for a session
def session_level(session, level):
      result = "N"
      # loop over attributevalues; if any exist with attribute_id == "SessionsbyExperienceLevel", and value == level
      for x in session["attributevalues"][:]:
        if "attribute_id" in x: 
            if x["attribute_id"]=="SessionsbyExperienceLevel":
                if x["value"]== level:
                    result="Y"
                    break # no point in continuing if we have found what we are looking for
      return result

This cell runs through all three level values and creates a new column in the data frame for each level. It will set a Y or N for each session in the new columns, depending on whether session is associated with the level, or not.

In [56]:
#set flags for designations
levels = ['Intermediate', 'Beginner', 'Advanced']
for l in levels:
   ss[l] = ss.apply(session_level, args=(l,), axis=1) 
print("Assigned Level Flags (Advanced, Intermediate, Beginner)")

Assigned Level Flags (Advanced, Intermediate, Beginner)


Next we will derive values for a new column 'All' that indicates whether a session has been associated with all levels - even though I am not sure what exactly that means.

In [59]:
def isAll(session):
    return 'Y' if session['Beginner'] == 'Y'  and session['Intermediate'] == 'Y' and session['Advanced'] == 'Y'else 'N'

ss['All'] = ss.apply( isAll, axis=1) 

In [60]:
ss[['Intermediate', 'Beginner', 'Advanced', 'All']].apply(pd.value_counts).fillna(0)

,Intermediate,Beginner,Advanced,All
Y,3224,2880,2444,2118
N,612,956,1392,1718


### Track
All sessions are assigned to one or more tracks. These tracks are categories that help attendees identify and assess sessions. Some examples of tracks are: Core Java Platform, Development Tools, Oracle Cloud Platform, MySQL, Containers, Serverless, and Cloud, Emerging Technologies, Modern Web, Application Development, Infrastructure Technologies (Data Center).

Depending on whether a session originates from the Oracle OpenWorld or CodeOne catalog, the track(s) are found in the nested object *attributevalues* under the attribute_id *CodeOneTracks* or just *Track*.

The next function is created to return a String array for a session with all the tracks associated with the session.

In [61]:
#function to derive track flag for a session
def session_track(session):
      result = ['abc']      
      # loop over attributevalues; if any exist with attribute_id == "SessionsbyExperienceLevel", and value == level
      for x in session["attributevalues"][:]:
        if "attribute_id" in x: 
            if x["attribute_id"]=="CodeOneTracks":
               result.append( x["value"])
            if x["attribute_id"]=="Track":               
               result.append( x["value"])
      del result[0]   
      return result

The next cell uses the function `session_track` to produce the value for each session for the track. The cell then prints out the tracks for a sample of sessions.

In [68]:
# add column track with a value derived from the session record
ss["track"] = ss.apply(session_track,  axis=1) 
print("Assigned Track")
ss[['title','catalog','track']].tail(10)

Assigned Track


,title,catalog,track
3826,The Java Platform Module System in Action: Bui...,codeone,"[Core Java Platform, Development Tools]"
3827,Using the MySQL Binary Log as a Change Stream,codeone,"[Oracle Cloud Platform, - MySQL, MySQL]"
3828,Clouds in My Coffee: Java on Mobile for iOS an...,codeone,"[Containers, Serverless, and Cloud, Java Ecosy..."
3829,Getting Started with Ethereum Private Blockchain,codeone,[Emerging Technologies]
3830,GraphQL in the Java World: Let’s Go for a Dive,codeone,"[Emerging Technologies, Modern Web]"
3831,"Java 5, 6, 7, 8, 9, 10, 11: What Did You Miss?",codeone,[Core Java Platform]
3832,Kubernetes for Java Developers,codeone,"[Containers, Serverless, and Cloud, Developmen..."
3833,Let's Start Java11 from the Beginning with JShell,codeone,[Core Java Platform]
3834,MySQL 8.0 Document Store: Everything You Need ...,codeone,"[Oracle Cloud Platform, - MySQL, MySQL]"
3835,Preventing Errors Before They Happen: The Chec...,codeone,[Development Tools]


### Number of Speakers per Session
The number of speakers making an appearance in a session is another fact that while not readily available is also hiding in our data frame. We will turn this piece of information into an explicit feature. The next cell counts the number of elements in the  participants array - and assigns it to the speaker_count attribute of each session record.

In [74]:
#set number of speakers for each session by taking the length of the list in the participants column
ss["speaker_count"] = ss['participants'].apply(lambda x: len(x))
# list the values in the new speaker_count column and the number of occurrences. 12 participants in one session?
ss['speaker_count'].value_counts()

1     1665
2     1117
3      793
4      143
5       66
8       20
6       18
7        8
11       2
12       2
10       2
Name: speaker_count, dtype: int64

### Number of instances per session
Most sessions are scheduled just once. However, some sessions are executed multiple times. This can be derived from the *times* column in the data frame, simply by taking the number of elements in the JSON array in that column.

The next cell adds a column to the data frame, with for each session the number of times that session takes place.

In [91]:
#set number of instancesfor each session by taking the length of the list in the times column
ss["instance_count"] = ss['times'].apply(lambda x: len(x) if x else None)
ss['instance_count'].value_counts()

1.0    1685
2.0      39
4.0       9
3.0       5
8.0       1
Name: instance_count, dtype: int64

### Session Room Capacity


In [182]:
#set the room capacity based on the capacity of the room of the first entry in the times list
# note: when the session is scheduled multiple times, not all rooms may have the same capacity; that detail gets lost
ss["room_capacity"]= ss['times'].apply(lambda x:x[0]['capacity'] if x else None)
#likewise derive day, time and room - from the first occurrence of the session
ss["day"]= ss['times'].apply(lambda x:x[0]['dayName'] if x else None)
ss["room"]= ss['times'].apply(lambda x:x[0]['room'] if x else None)
ss["time"]= ss['times'].apply(lambda x:x[0]['time'] if x else None)
ss.head(5)

,abstract,attributevalues,catalog,code,event,eventCode,files,length,participants,sponsors,...,Beginner,Advanced,All,track,speaker_count,instance_count,room_capacity,day,room,time
0,What if you could benefit from new database qu...,"[{'value': 'Beginner', 'attributevalue_id': '1...",oow,TRN4070,Oracle OpenWorld,oow18,"[{'fileId': '1540594881063001PjdE', 'filename'...",45.0,[{'speakerId': '14574694377280015QNA_150393630...,None,...,Y,Y,N,"[- Application Development, Oracle Cloud Platf...",3,1.0,308,Wednesday,Moscone West - Room 3009,11:15
1,How does a real-world performance engineer int...,"[{'value': 'Beginner', 'attributevalue_id': '1...",oow,TRN4026,Oracle OpenWorld,oow18,"[{'fileId': '1540053090863001JeFM', 'filename'...",45.0,[{'speakerId': '145746969078000151fg_150393630...,None,...,Y,Y,N,"[- Application Development, Oracle Cloud Platf...",2,1.0,313,Monday,Moscone West - Room 3004,15:45
2,Understand the tools and techniques available ...,"[{'value': 'Beginner', 'attributevalue_id': '1...",oow,TRN4567,Oracle OpenWorld,oow18,"[{'fileId': '1539988618000001VcCe', 'filename'...",45.0,[{'speakerId': '1468532302119001DtNf_150393630...,None,...,Y,Y,N,"[SMB Solutions, Oracle Cloud Platform, - Cloud...",1,1.0,312,Monday,Moscone West - Room 3006,11:30
3,In order to derive value from the Internet of ...,"[{'value': 'Product Training Session', 'attrib...",oow,TRN6363,Oracle OpenWorld,oow18,"[{'fileId': '1539982921144001RL4h', 'filename'...",45.0,[{'speakerId': '1500580326390001qTtd_150393630...,None,...,N,N,N,[],1,1.0,173,Monday,Moscone West - Room 3000,12:30
4,Oracle Linux powers tens of thousands of enter...,"[{'value': 'Beginner', 'attributevalue_id': '1...",oow,TRN5828,Oracle OpenWorld,oow18,"[{'fileId': '1540152941675001kZHw', 'filename'...",45.0,[{'speakerId': '14574694616420015a7T_150393630...,None,...,Y,Y,N,"[Your Cloud Transformation Roadmap, - Upgradin...",2,1.0,50000,Monday,Moscone South - Room 154,10:30


## Deduplicate
Some sessions are included in the catalog for both events - Oracle OpenWorld and CodeOne - even though they are associated with one of the two. The exact same session - with only a different value for attribute catalog - occurs twice in our data set for these sessions. We should get rid of duplicates. However, we should not do so before we capture the fact that a session is part of the catalogs of both events in the record that is retained. 

Note: there some sessions that occur multiple times in the data set but are not included in both event catalogs.

Let's first see how many events are part of both events' catalogs. 

In [183]:
#All three approaches do the work of finding all rows with code values that occur more than once

#ss[ss['code'].duplicated(keep=False)]
#ss.groupby('code').filter(lambda x: len(x) > 1)

counts = ss['code'].value_counts()
duplicates = ss[ss['code'].isin(counts.index[counts > 1])]

duplicates.sort_values('code').head(10)


,abstract,attributevalues,catalog,code,event,eventCode,files,length,participants,sponsors,...,Beginner,Advanced,All,track,speaker_count,instance_count,room_capacity,day,room,time
147,Microservices are independent—sure. Complex tr...,"[{'value': 'Intermediate', 'attributevalue_id'...",oow,BOF4977,Oracle OpenWorld,oow18,"[{'fileId': '1540345180625001DgUR', 'filename'...",45.0,[{'speakerId': '14574694420080015I8e_150393630...,None,...,N,N,N,"[Containers, Serverless, and Cloud, Infrastruc...",1,1.0,303,Tuesday,Moscone West - Room 2006,19:30
151,Microservices are independent—sure. Complex tr...,"[{'value': 'Intermediate', 'attributevalue_id'...",codeone,BOF4977,Oracle OpenWorld,oow18,"[{'fileId': '1540345180625001DgUR', 'filename'...",45.0,[{'speakerId': '14574694420080015I8e_150393630...,None,...,N,N,N,"[Containers, Serverless, and Cloud, Infrastruc...",1,1.0,303,Tuesday,Moscone West - Room 2006,19:30
148,"Two years ago, after a painful history of wate...","[{'value': 'Beginner', 'attributevalue_id': '1...",oow,BOF5232,Oracle OpenWorld,oow18,"[{'fileId': '1540253608259001DfiR', 'filename'...",45.0,[{'speakerId': '14574694740030015Ufr_150393630...,None,...,Y,Y,N,"[DevOps and Pipelines, Development Tools, Java...",4,1.0,283,Monday,Moscone West - Room 2024,20:30
166,"Two years ago, after a painful history of wate...","[{'value': 'Beginner', 'attributevalue_id': '1...",codeone,BOF5232,Oracle OpenWorld,oow18,"[{'fileId': '1540253608259001DfiR', 'filename'...",45.0,[{'speakerId': '14574694740030015Ufr_150393630...,None,...,Y,Y,N,"[DevOps and Pipelines, Development Tools, Java...",4,1.0,283,Monday,Moscone West - Room 2024,20:30
149,The migration from monolith to microservices h...,"[{'value': 'Beginner', 'attributevalue_id': '1...",oow,BOF5714,Oracle OpenWorld,oow18,"[{'fileId': '1540337005562001JXhu', 'filename'...",45.0,[{'speakerId': '1526055148128001q2D3_150393630...,None,...,Y,N,N,"[Containers, Serverless, and Cloud, Infrastruc...",1,1.0,298,Tuesday,Moscone West - Room 2001,19:30
167,The migration from monolith to microservices h...,"[{'value': 'Beginner', 'attributevalue_id': '1...",codeone,BOF5714,Oracle OpenWorld,oow18,"[{'fileId': '1540337005562001JXhu', 'filename'...",45.0,[{'speakerId': '1526055148128001q2D3_150393630...,None,...,Y,N,N,"[Containers, Serverless, and Cloud, Infrastruc...",1,1.0,298,Tuesday,Moscone West - Room 2001,19:30
146,Data is the essential fuel to every analytic p...,"[{'value': 'Beginner', 'attributevalue_id': '1...",oow,BOF6086,Oracle OpenWorld,oow18,"[{'fileId': '1540230846859001JH0Y', 'filename'...",45.0,[{'speakerId': '14574694443730025zo5_150393630...,[],...,Y,N,N,"[Database, Big Data, and Data Science, Infrast...",2,1.0,280,Monday,Moscone West - Room 2003,20:30
150,Data is the essential fuel to every analytic p...,"[{'value': 'Beginner', 'attributevalue_id': '1...",codeone,BOF6086,Oracle OpenWorld,oow18,"[{'fileId': '1540230846859001JH0Y', 'filename'...",45.0,[{'speakerId': '14574694443730025zo5_150393630...,[],...,Y,N,N,"[Database, Big Data, and Data Science, Infrast...",2,1.0,280,Monday,Moscone West - Room 2003,20:30
361,Since Oracle announced support for some of its...,"[{'value': 'Beginner', 'attributevalue_id': '1...",codeone,BUS1224,Oracle OpenWorld,oow18,"[{'fileId': '1537442351848001mIzP', 'filename'...",45.0,[{'speakerId': '14574695992920015zm9_150393630...,None,...,Y,N,N,"[Database, Big Data, and Data Science, Your Cl...",1,1.0,305,Thursday,Moscone West - Room 3018,12:00
290,Since Oracle announced support for some of its...,"[{'value': 'Beginner', 'attributevalue_id': '1...",oow,BUS1224,Oracle OpenWorld,oow18,"[{'fileId': '1537442351848001mIzP', 'filename'...",45.0,[{'speakerId': '14574695992920015zm9_150393630...,None,...,Y,N,N,"[Database, Big Data, and Data Science, Your Cl...",1,1.0,305,Thursday,Moscone West - Room 3018,12:00


In [184]:
#are duplicates indeed associated with both events?
# some are - but not all of them:
duplicates.loc[duplicates['code']=='BOF4977']


,abstract,attributevalues,catalog,code,event,eventCode,files,length,participants,sponsors,...,Beginner,Advanced,All,track,speaker_count,instance_count,room_capacity,day,room,time
147,Microservices are independent—sure. Complex tr...,"[{'value': 'Intermediate', 'attributevalue_id'...",oow,BOF4977,Oracle OpenWorld,oow18,"[{'fileId': '1540345180625001DgUR', 'filename'...",45.0,[{'speakerId': '14574694420080015I8e_150393630...,None,...,N,N,N,"[Containers, Serverless, and Cloud, Infrastruc...",1,1.0,303,Tuesday,Moscone West - Room 2006,19:30
151,Microservices are independent—sure. Complex tr...,"[{'value': 'Intermediate', 'attributevalue_id'...",codeone,BOF4977,Oracle OpenWorld,oow18,"[{'fileId': '1540345180625001DgUR', 'filename'...",45.0,[{'speakerId': '14574694420080015I8e_150393630...,None,...,N,N,N,"[Containers, Serverless, and Cloud, Infrastruc...",1,1.0,303,Tuesday,Moscone West - Room 2006,19:30


In [185]:
# find all sessions that occur in both catalogs: set their catalog attribute to both
# set catalog="both" if session in duplicates with a different catalog value than the session's own catalog value
doubleCatalogSessions = pd.merge(ss, duplicates, on=['code'], how='inner').query('catalog_y != catalog_x')
doubleCatalogSessions[['code','catalog_x', 'catalog_y']] .head(20)



,code,catalog_x,catalog_y
45,TRN5793,oow,codeone
47,TRN5793,oow,codeone
48,TRN5793,codeone,oow
50,TRN5793,codeone,oow
53,TRN5793,oow,codeone
55,TRN5793,oow,codeone
56,TRN5793,codeone,oow
58,TRN5793,codeone,oow
137,TRN4084,oow,codeone
139,TRN4084,oow,codeone


In [186]:
# all sessions in doubleCatalogSessions occur in both oow and code one session catalog
# time to update column catalog for all sessions in ss that have a code that occurs in doubleCatalogSessions['code']
ss.loc[ss['code'].isin(doubleCatalogSessions['code']),'catalog']='both'
ss.sort_values('catalog').head(50)

,abstract,attributevalues,catalog,code,event,eventCode,files,length,participants,sponsors,...,Beginner,Advanced,All,track,speaker_count,instance_count,room_capacity,day,room,time
1169,In this session get a practical introduction t...,"[{'value': 'Beginner', 'attributevalue_id': '1...",both,HOL1706,Oracle OpenWorld,oow18,"[{'fileId': '1540332651013001QC7t', 'filename'...",60.0,[{'speakerId': '14574694383250055jg8_150393630...,NaN,...,Y,Y,N,"[Hands-on Labs (HOLs), Oracle Cloud Platform, ...",2,1.0,40,Tuesday,Marriott Marquis (Yerba Buena Level) - Salon 9A,11:15
1157,In this session learn to develop and deploy a ...,"[{'value': 'Beginner', 'attributevalue_id': '1...",both,HOL6325,Oracle OpenWorld,oow18,"[{'fileId': '1539975363677001dU4z', 'filename'...",60.0,[{'speakerId': '145746947297400155og_150393630...,NaN,...,Y,Y,N,"[- Application Development, Hands-on Labs (HOL...",3,3.0,48,Monday,Marriott Marquis (Yerba Buena Level) - Salon 3/4,15:45
1158,In this session learn about the latest feature...,"[{'value': 'Beginner', 'attributevalue_id': '1...",both,HOL6328,Oracle OpenWorld,oow18,"[{'fileId': '1543251215909001VEhl', 'filename'...",60.0,[{'speakerId': '14574694354950015iFv_150393630...,NaN,...,Y,Y,N,"[- Application Development, Hands-on Labs (HOL...",2,1.0,48,Thursday,Marriott Marquis (Yerba Buena Level) - Salon 3/4,12:00
1164,One of the marquee features of Oracle Applicat...,"[{'value': 'Beginner', 'attributevalue_id': '1...",both,HOL6326,Oracle OpenWorld,oow18,None,60.0,[{'speakerId': '14574694670000025Uc7_150393630...,NaN,...,Y,Y,N,"[- Application Development, Hands-on Labs (HOL...",2,1.0,48,Wednesday,Marriott Marquis (Yerba Buena Level) - Salon 3/4,09:30
1167,In this session learn how to use the MySQL Doc...,"[{'value': 'Beginner', 'attributevalue_id': '1...",both,HOL1703,Oracle OpenWorld,oow18,"[{'fileId': '1540332488585001Jlrb', 'filename'...",60.0,[{'speakerId': '14574694383250055jg8_150393630...,NaN,...,Y,Y,N,"[Hands-on Labs (HOLs), Oracle Cloud Platform, ...",2,1.0,40,Monday,Marriott Marquis (Yerba Buena Level) - Salon 9A,12:15
1168,In this session discover Oracle NoSQL Database...,"[{'value': 'Intermediate', 'attributevalue_id'...",both,HOL6302,Oracle OpenWorld,oow18,None,60.0,[{'speakerId': '14574694760770015GOv_150393630...,NaN,...,N,N,N,"[- Internet of Things (IoT), Hands-on Labs (HO...",2,2.0,48,Tuesday,Marriott Marquis (Yerba Buena Level) - Salon 1/2,17:15
2337,"During this tutorial, you will learn how to us...","[{'value': 'Beginner', 'attributevalue_id': '1...",both,TUT6303,Oracle OpenWorld,oow18,"[{'fileId': '1541454340556001bnq5', 'filename'...",120.0,[{'speakerId': '1464284608830001XvOn_150393630...,None,...,Y,Y,N,"[Oracle Cloud Platform, - MySQL, MySQL]",1,1.0,301,Tuesday,Moscone West - Room 2016,08:45
1183,In this lab learn to write an asynchronous ser...,"[{'value': 'Beginner', 'attributevalue_id': '1...",both,HOL4799,Oracle OpenWorld,oow18,"[{'fileId': '1539905700801001DGTY', 'filename'...",60.0,[{'speakerId': '14574694373590015wCR_150393630...,NaN,...,Y,Y,N,"[Hands-on Labs (HOLs), Database, Big Data, and...",3,1.0,48,Wednesday,Marriott Marquis (Yerba Buena Level) - Salon 3/4,14:15
1192,Oracle Application Express is a low-code devel...,"[{'value': 'Beginner', 'attributevalue_id': '1...",both,HOL6327,Oracle OpenWorld,oow18,None,60.0,[{'speakerId': '14574694670000025Uc7_150393630...,NaN,...,Y,Y,N,"[- Application Development, Hands-on Labs (HOL...",2,1.0,48,Thursday,Marriott Marquis (Yerba Buena Level) - Salon 3/4,09:00
1204,<b>PLEASE NOTE: YOU MUST BRING YOUR OWN LAPTOP...,"[{'value': 'Beginner', 'attributevalue_id': '1...",both,HOL6311,Oracle OpenWorld,oow18,None,60.0,[{'speakerId': '14574694693940015ObX_150393630...,NaN,...,Y,N,N,"[Hands-on Labs (HOLs), SMB Solutions, Infrastr...",4,4.0,40,Monday,Marriott Marquis (Yerba Buena Level) - Salon 9B,17:15


In [187]:
#Drop duplicates - identifying rows by their code
ss.drop_duplicates(subset=['code'], keep='first', inplace=True)

In [188]:
#hpw many sessions appear in both catalogs?
ss['catalog'].value_counts()

oow        1434
codeone     477
both        141
Name: catalog, dtype: int64

## Persist Pandas Dataframe - as single, consolidated, reshaped, enriched JSON file

One thing we want to be able to do with the data we gather, is to persist it for future use. We could store the data in a NoSQL database, a cloud storage service or simply as a local file. For now, let's do the latter: store the cleansed and reshaped data in a local JSON file for further enrichment, visualization and machine learning purposes.

In [189]:
ss.to_json("oow2018-sessions-wrangled.json", force_ascii=False)

In [190]:
dubss = pd.read_json("oow2018-sessions-wrangled.json")
dubss.head(20)

,Advanced,All,Beginner,Groundbreaker Ambassador,Intermediate,JavaOne Rockstar,Oracle ACE Director,Oracle Java Champion,abstract,attributevalues,...,room,room_capacity,speaker_count,sponsors,time,times,title,title_sort,track,type
0,Y,N,Y,N,Y,N,N,N,What if you could benefit from new database qu...,"[{'value': 'Beginner', 'attributevalue_id': '1...",...,Moscone West - Room 3009,308.0,3,None,11:15,"[{'sessionTimeID': '1532616475210001usPa', 'se...",A Database Proxy for Transparent High Availabi...,adatabaseproxyfortransparenthi,"[- Application Development, Oracle Cloud Platf...",Product Training Session
1,Y,N,Y,N,Y,N,N,N,How does a real-world performance engineer int...,"[{'value': 'Beginner', 'attributevalue_id': '1...",...,Moscone West - Room 3004,313.0,2,None,15:45,"[{'sessionTimeID': '1531945554973001z4Hi', 'se...",A Day in the Life of a Real-World Performance ...,adayinthelifeofarealworldperfo,"[- Application Development, Oracle Cloud Platf...",Product Training Session
10,Y,N,Y,N,Y,Y,N,N,In this session learn how to make client data ...,"[{'value': 'Beginner', 'attributevalue_id': '1...",...,Moscone West - Room 3002,314.0,1,None,15:45,"[{'sessionTimeID': '1535610677698001BB58', 'se...",Providing High Availability of Data by Using C...,providinghighavailabilityofdat,[],Product Training Session
100,N,N,N,N,N,N,N,N,In this session learn how to transfer your st...,"[{'value': 'Product Training Session', 'attrib...",...,The Exchange @ Moscone South - OU Mini Theater,75.0,1,None,16:25,"[{'sessionTimeID': '15381128815670018LA1', 'se...",How to Transfer Storage to Oracle Storage,howtotransferstoragetooraclest,[],Product Training Session
1000,Y,N,N,N,Y,N,N,Y,With Java Flight Recorder (JFR) and Java Missi...,"[{'value': 'Intermediate', 'attributevalue_id'...",...,Moscone West - Room 2024,283.0,1,None,11:30,"[{'sessionTimeID': '15355059853370011rXS', 'se...",Three Productive Ways to Use Open Source Java ...,threeproductivewaystouseopenso,"[Developer Community, Development Tools, Java ...",Developer Session
1002,N,N,N,N,Y,N,N,N,Failure is not an option in services that need...,"[{'value': 'Intermediate', 'attributevalue_id'...",...,Moscone West - Room 2010,243.0,2,None,12:00,"[{'sessionTimeID': '1535566931714001Bodf', 'se...",Three Resilience Patterns Out of the Box with ...,threeresiliencepatternsoutofth,[Java Server-Side Development and Microservices],Developer Session
1005,N,N,Y,N,N,N,Y,N,One of the most important factors influencing ...,"[{'value': 'Beginner', 'attributevalue_id': '1...",...,Moscone West - Room 2005,280.0,3,None,14:00,"[{'sessionTimeID': '1535417091221001BkmK', 'se...",Tracking Spare Parts with Blockchain Featuring...,trackingsparepartswithblockcha,"[Containers, Serverless, and Cloud, Database, ...",Developer Session
1006,N,N,N,N,Y,N,N,N,The long-awaited Transport Layer Security (TLS...,"[{'value': 'Intermediate', 'attributevalue_id'...",...,Moscone West - Room 2004,285.0,2,None,11:30,"[{'sessionTimeID': '1536180227506001xIyD', 'se...",Transport Layer Security (TLS) v1.3 support in...,transportlayersecuritytlsv13su,[Core Java Platform],Developer Session
1007,N,N,N,N,Y,Y,N,N,"Apart from the Java heap and the metaspace, wh...","[{'value': 'Intermediate', 'attributevalue_id'...",...,Moscone West - Room 2008,303.0,1,None,09:00,"[{'sessionTimeID': '1535488178716001Bb9j', 'se...",Troubleshooting Native Memory Leaks in Java Ap...,troubleshootingnativememorylea,"[Containers, Serverless, and Cloud, Core Java ...",Developer Session
1008,N,N,Y,N,N,N,N,N,Planning to build microservices? The best prac...,"[{'value': 'Beginner', 'attributevalue_id': '1...",...,Moscone West - Room 2003,280.0,1,[],13:30,"[{'sessionTimeID': '15355648119050012ZTw', 'se...",Build a 12-Factor Microservice with MicroProfile,builda12factormicroservicewith,"[Java Ecosystem, Java Server-Side Development ...",Developer Session


If and when the previous cell lists 20 sessions, the data lake has been populated with the consolidated file *oow2018-sessions-wrangled.json* with all sessions - cleansed, deduplicated, enriched and ready for further processing.